# Modele de regresie: Wisconsin Breast Cancer
Tătaru Dragoș-Cătălin, _grupa 10LF383_

In [1]:
import numpy as np
import pandas as pd

from IPython.display import display, HTML
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate,cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold


import sklearn
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.linear_model import Lasso, BayesianRidge, ElasticNet
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import uniform as sp_rand
import scipy.stats as stats

# Loading the data set

In [2]:
heads=['Lymph_node','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave_points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave_points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave_points_worst','symmetry_worst','fractal_dimension_worst','Tumor_size','Time']

# Replacing missing data types with NAN
missingValues = ["n/a", "na", "--"," ","?"]
data_wisconsin = pd.read_csv("./Datasets/Wisconsin Breast Cancer/r_wpbc.data",sep=',',names=heads,na_values=missingValues)

display(data_wisconsin)

,Lymph_node,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst,Tumor_size,Time
0,5,18.02,27.60,117.50,1013.0,0.09489,0.10360,0.10860,0.07055,0.1865,...,139.70,1436.0,0.11950,0.1926,0.3140,0.11700,0.2677,0.08113,5.0,31
1,2,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890,3.0,61
2,0,21.37,17.44,137.50,1373.0,0.08836,0.11890,0.12550,0.08180,0.2333,...,159.10,1949.0,0.11880,0.3449,0.3414,0.20320,0.4334,0.09067,2.5,116
3,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300,2.0,123
4,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678,3.5,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,21,14.72,25.26,99.28,657.5,0.11740,0.21120,0.17290,0.09465,0.2079,...,111.60,814.8,0.14640,0.5352,0.5655,0.19740,0.3778,0.11320,1.7,3
190,2,22.52,21.92,146.90,1597.0,0.07592,0.09162,0.06862,0.06367,0.1728,...,162.10,1902.0,0.08191,0.1319,0.1056,0.09378,0.2061,0.05788,6.0,10
191,0,15.44,31.18,101.00,740.4,0.09399,0.10620,0.13750,0.06500,0.1735,...,112.60,929.0,0.12720,0.2362,0.2975,0.12860,0.2914,0.08024,1.5,8
192,0,17.17,29.19,110.00,915.3,0.08952,0.06655,0.06583,0.05068,0.1793,...,132.50,1295.0,0.12610,0.1572,0.2141,0.09520,0.3362,0.06033,3.7,12


# Checking for missing data

In [3]:
display(data_wisconsin.info())

# Afisam numarul de missind data in functie de coloana
df=pd.DataFrame(data_wisconsin.isnull().sum())

# pentru a putea afisa toate coloanele nu doar limita pd
with pd.option_context('display.max_rows', None):display(df)

# Afisam numarul de missind data in total
print (data_wisconsin.isnull().sum().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Lymph_node               194 non-null    int64  
 1   radius_mean              194 non-null    float64
 2   texture_mean             194 non-null    float64
 3   perimeter_mean           194 non-null    float64
 4   area_mean                194 non-null    float64
 5   smoothness_mean          194 non-null    float64
 6   compactness_mean         194 non-null    float64
 7   concavity_mean           194 non-null    float64
 8   concave_points_mean      194 non-null    float64
 9   symmetry_mean            194 non-null    float64
 10  fractal_dimension_mean   194 non-null    float64
 11  radius_se                194 non-null    float64
 12  texture_se               194 non-null    float64
 13  perimeter_se             194 non-null    float64
 14  area_se                  1

None

,0
Lymph_node,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave_points_mean,0
symmetry_mean,0


0


# Splitting the dataset

In [4]:
newHead=data_wisconsin.columns

X = data_wisconsin.values[:, :32]
X = pd.DataFrame(X, columns=newHead[0:32])
y = data_wisconsin.values[:, -1]
y = pd.DataFrame(y, columns=['Time'])

display(X)
display(y)

,Lymph_node,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst,Tumor_size
0,5.0,18.02,27.60,117.50,1013.0,0.09489,0.10360,0.10860,0.07055,0.1865,...,37.08,139.70,1436.0,0.11950,0.1926,0.3140,0.11700,0.2677,0.08113,5.0
1,2.0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890,3.0
2,0.0,21.37,17.44,137.50,1373.0,0.08836,0.11890,0.12550,0.08180,0.2333,...,20.98,159.10,1949.0,0.11880,0.3449,0.3414,0.20320,0.4334,0.09067,2.5
3,0.0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300,2.0
4,0.0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,21.0,14.72,25.26,99.28,657.5,0.11740,0.21120,0.17290,0.09465,0.2079,...,31.97,111.60,814.8,0.14640,0.5352,0.5655,0.19740,0.3778,0.11320,1.7
190,2.0,22.52,21.92,146.90,1597.0,0.07592,0.09162,0.06862,0.06367,0.1728,...,24.81,162.10,1902.0,0.08191,0.1319,0.1056,0.09378,0.2061,0.05788,6.0
191,0.0,15.44,31.18,101.00,740.4,0.09399,0.10620,0.13750,0.06500,0.1735,...,41.48,112.60,929.0,0.12720,0.2362,0.2975,0.12860,0.2914,0.08024,1.5
192,0.0,17.17,29.19,110.00,915.3,0.08952,0.06655,0.06583,0.05068,0.1793,...,36.66,132.50,1295.0,0.12610,0.1572,0.2141,0.09520,0.3362,0.06033,3.7


,Time
0,31.0
1,61.0
2,116.0
3,123.0
4,27.0
...,...
189,3.0
190,10.0
191,8.0
192,12.0


# Lasso Regression

In [5]:
pipe = Pipeline([('lss', Lasso(max_iter=50000,))])

parameter_grid = { 'lss__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}
k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y, cv=k_fold,scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_lasso = pd.DataFrame(scores)

pd_scores_lasso.insert(0, "ModelName", ['Lasso_1', 'Lasso_2','Lasso_3', 'Lasso_4','Lasso_5']) 
display(pd_scores_lasso)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,Lasso_1,2.658133,0.002151,-24.623239,-26.009587,-844.521120,-975.253244,-21.000266,-23.178645
1,Lasso_2,2.434922,0.002159,-34.338245,-21.332367,-1572.385222,-733.581408,-35.579730,-17.358543
2,Lasso_3,2.405780,0.001982,-23.473062,-25.968226,-786.519748,-983.090856,-22.371202,-22.331695
3,Lasso_4,2.407281,0.001962,-29.677703,-24.709081,-1235.574349,-883.383449,-26.749633,-21.680006
4,Lasso_5,2.953219,0.002047,-27.873625,-25.086161,-1180.815428,-903.097791,-24.621374,-21.351845


# Ridge Regression

In [6]:
pipe = Pipeline([('rid', linear_model.Ridge())])
parameter_grid = { 'rid__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}
k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y, cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_ridge = pd.DataFrame(scores)

pd_scores_ridge.insert(0, "ModelName", ['Ridge_1', 'Ridge_2','Ridge_3', 'Ridge_4','Ridge_5']) 
display(pd_scores_ridge)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,Ridge_1,1.311014,0.002632,-24.898257,-25.269479,-976.055392,-937.928096,-17.784002,-22.707130
1,Ridge_2,1.326184,0.002123,-26.018289,-25.342688,-974.051312,-914.131956,-24.582613,-23.523375
2,Ridge_3,1.220302,0.002078,-26.206267,-24.226489,-969.396537,-897.236202,-21.734636,-20.331969
3,Ridge_4,1.201834,0.002119,-29.286199,-24.081456,-1206.847427,-844.922703,-26.692713,-22.620742
4,Ridge_5,1.204178,0.002160,-29.718000,-23.382099,-1329.112324,-832.506117,-25.616611,-19.925832


# Elastic Net Regression

In [7]:
pipe = Pipeline([('enr', linear_model.ElasticNet(max_iter=50000))])

parameter_grid = { 'enr__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float),'enr__l1_ratio':np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}
k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=200)
scores = cross_validate(grid_search, X, y, cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_net_reg = pd.DataFrame(scores)

pd_scores_net_reg.insert(0, "ModelName", ['ElasticNet_1', 'ElasticNet_2','ElasticNet_3', 'ElasticNet_4','ElasticNet_5']) 
display(pd_scores_net_reg)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,ElasticNet_1,4.634880,0.002065,-22.230744,-26.299498,-782.986927,-984.811862,-17.472403,-23.609699
1,ElasticNet_2,4.050608,0.001982,-26.999517,-25.218156,-1014.626786,-937.745686,-24.655998,-22.785140
2,ElasticNet_3,4.009923,0.002048,-30.016421,-23.613279,-1258.698178,-849.175842,-30.538867,-21.632529
3,ElasticNet_4,4.416899,0.001984,-30.304008,-24.503652,-1235.579188,-876.015207,-27.103604,-21.781495
4,ElasticNet_5,4.401627,0.002168,-27.379070,-24.934534,-1062.734305,-923.586208,-23.607289,-22.215202


# SGD Regression

In [8]:
pipe = Pipeline([('scaler', StandardScaler()),('sgr', linear_model.SGDRegressor(max_iter=10000))])

parameter_grid = {
                  'sgr__loss': ['squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive'],
                  'sgr__l1_ratio':  np.linspace(start=0, stop=1),
                  'sgr__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float)
                 }

k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y.to_numpy().ravel(), cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_sgdr = pd.DataFrame(scores)

pd_scores_sgdr.insert(0, "ModelName", ['SGD_1', 'SGD_2','SGD_3', 'SGD_4','SGD_5']) 
display(pd_scores_sgdr)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,SGD_1,2.253189,0.001293,-29.052591,-23.351064,-1336.530327,-943.173339,-21.945188,-17.682262
1,SGD_2,2.466469,0.001042,-28.228015,-23.751747,-1122.731932,-840.816080,-27.666805,-21.202279
2,SGD_3,2.066876,0.001050,-28.890331,-28.880015,-1291.130244,-1332.292813,-25.567595,-22.437690
3,SGD_4,1.981834,0.001169,-26.464000,-26.452579,-1103.618479,-1087.658355,-21.560624,-22.788809
4,SGD_5,2.281567,0.001195,-35.273627,-26.949285,-1813.925908,-1135.633542,-26.117784,-23.573862


# LassoLars Regression

In [9]:
pipe = Pipeline([('lar',linear_model.LassoLars())])
parameter_grid = { 'lar__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}

k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y.to_numpy().ravel(), cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_laso = pd.DataFrame(scores)

pd_scores_laso.insert(0, "ModelName", ['LassoLars_1', 'LassoLars_2','LassoLars_3', 'LassoLars_4','LassoLars_5']) 
display(pd_scores_laso)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,LassoLars_1,1.203559,0.001331,-21.759573,-26.765715,-731.224951,-1000.811829,-19.518799,-24.652873
1,LassoLars_2,1.128938,0.001065,-26.614731,-23.585664,-1057.587351,-830.462993,-24.580999,-21.431295
2,LassoLars_3,1.136053,0.001023,-31.743132,-23.344003,-1297.248725,-835.691937,-28.508684,-20.199749
3,LassoLars_4,1.066901,0.001015,-24.484044,-24.421827,-850.373272,-875.700493,-23.364700,-22.133872
4,LassoLars_5,1.082977,0.001025,-30.705717,-23.790155,-1407.177860,-803.484466,-27.373276,-21.713302


In [10]:
data_model_report=pd.DataFrame()
data_model_report=data_model_report.append([pd_scores_lasso,pd_scores_ridge,pd_scores_net_reg,pd_scores_sgdr,pd_scores_laso],ignore_index=True)
data_model_report.insert(1, " SearchStrategy ",'RandomizedSearchCV')

# Generarea raportului

In [11]:
data_model_report['test_neg_mean_absolute_error'] = -data_model_report[['test_neg_mean_absolute_error']]
data_model_report[['train_neg_mean_absolute_error']] = -data_model_report[['train_neg_mean_absolute_error']]
data_model_report[['test_neg_mean_squared_error']] = -data_model_report[['test_neg_mean_squared_error']]
data_model_report[['train_neg_mean_squared_error']] = -data_model_report[['train_neg_mean_squared_error']]
data_model_report[['test_neg_median_absolute_error']] = -data_model_report[['test_neg_median_absolute_error']]
data_model_report[['train_neg_median_absolute_error']] = -data_model_report[['train_neg_median_absolute_error']]

data_model_report.rename(columns={'test_neg_mean_absolute_error': 'test_mean_absolute_error',
                      'train_neg_mean_absolute_error':  'train_mean_absolute_error',
                      'test_neg_mean_squared_error':    'test_mean_squared_error',
                      'train_neg_mean_squared_error':   'train_mean_squared_error',
                      'test_neg_median_absolute_error': 'test_median_absolute_error', 
                      'train_neg_median_absolute_error':'train_median_absolute_error'}, inplace=True)

In [12]:
def highlight(s):
    is_max = s == s.max()
    is_min = s == s.min()
    styles = []
    for i in range(0, len(is_max)):
        if is_max[i]:
            styles.append('background-color: red')
        elif is_min[i]:
            styles.append('background-color: green')
        else:
            styles.append('')
    return styles

data_model_report_styled =data_model_report.style.apply(highlight, subset=['test_mean_absolute_error','train_mean_absolute_error','test_mean_squared_error','train_mean_squared_error','test_median_absolute_error','train_median_absolute_error'])

display(data_model_report_styled)

,ModelName,SearchStrategy,fit_time,score_time,test_mean_absolute_error,train_mean_absolute_error,test_mean_squared_error,train_mean_squared_error,test_median_absolute_error,train_median_absolute_error
0,Lasso_1,RandomizedSearchCV,2.658133,0.002151,24.623239,26.009587,844.521120,975.253244,21.000266,23.178645
1,Lasso_2,RandomizedSearchCV,2.434922,0.002159,34.338245,21.332367,1572.385222,733.581408,35.579730,17.358543
2,Lasso_3,RandomizedSearchCV,2.405780,0.001982,23.473062,25.968226,786.519748,983.090856,22.371202,22.331695
3,Lasso_4,RandomizedSearchCV,2.407281,0.001962,29.677703,24.709081,1235.574349,883.383449,26.749633,21.680006
4,Lasso_5,RandomizedSearchCV,2.953219,0.002047,27.873625,25.086161,1180.815428,903.097791,24.621374,21.351845
5,Ridge_1,RandomizedSearchCV,1.311014,0.002632,24.898257,25.269479,976.055392,937.928096,17.784002,22.707130
6,Ridge_2,RandomizedSearchCV,1.326184,0.002123,26.018289,25.342688,974.051312,914.131956,24.582613,23.523375
7,Ridge_3,RandomizedSearchCV,1.220302,0.002078,26.206267,24.226489,969.396537,897.236202,21.734636,20.331969
8,Ridge_4,RandomizedSearchCV,1.201834,0.002119,29.286199,24.081456,1206.847427,844.922703,26.692713,22.620742
9,Ridge_5,RandomizedSearchCV,1.204178,0.002160,29.718000,23.382099,1329.112324,832.506117,25.616611,19.925832


In [13]:
html_str = """
<h1><u>Wisconsin Breast Cancer</u></h1>
"""
html_str = html_str + (data_model_report_styled.render())
html_file = open("./Reports/wisconsin_reg_report.html", "w")
html_file.write(html_str)
html_file.close()